Untuk menjalankan notebook berikut, jika Anda belum melakukannya, Anda perlu menerapkan model yang menggunakan `text-embedding-ada-002` sebagai model dasar dan mengatur nama deployment di dalam file .env sebagai `AZURE_OPENAI_EMBEDDINGS_ENDPOINT`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Selanjutnya, kita akan memuat Indeks Embedding ke dalam Dataframe Pandas. Indeks Embedding disimpan dalam file JSON bernama `embedding_index_3m.json`. Indeks Embedding berisi Embedding untuk setiap transkrip YouTube hingga akhir Oktober 2023.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Selanjutnya, kita akan membuat sebuah fungsi bernama `get_videos` yang akan mencari Embedding Index berdasarkan query. Fungsi ini akan mengembalikan 5 video teratas yang paling mirip dengan query tersebut. Cara kerja fungsi ini adalah sebagai berikut:

1. Pertama, dibuat salinan dari Embedding Index.
2. Selanjutnya, Embedding untuk query dihitung menggunakan OpenAI Embedding API.
3. Lalu, dibuat kolom baru di Embedding Index yang disebut `similarity`. Kolom `similarity` berisi nilai cosine similarity antara Embedding query dan Embedding untuk setiap segmen video.
4. Setelah itu, Embedding Index difilter berdasarkan kolom `similarity`. Embedding Index hanya akan menampilkan video yang memiliki cosine similarity lebih besar atau sama dengan 0.75.
5. Terakhir, Embedding Index diurutkan berdasarkan kolom `similarity` dan 5 video teratas akan dikembalikan.


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Fungsi ini sangat sederhana, fungsi ini hanya mencetak hasil dari kueri pencarian.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Pertama, Embedding Index dimuat ke dalam Dataframe Pandas.
2. Selanjutnya, pengguna diminta untuk memasukkan sebuah kueri.
3. Kemudian fungsi `get_videos` dipanggil untuk mencari Embedding Index berdasarkan kueri tersebut.
4. Terakhir, fungsi `display_results` dipanggil untuk menampilkan hasilnya kepada pengguna.
5. Pengguna kemudian diminta untuk memasukkan kueri lain. Proses ini akan terus berlanjut sampai pengguna mengetik `exit`.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.id.png)

Anda akan diminta untuk memasukkan sebuah kueri. Masukkan kueri lalu tekan enter. Aplikasi akan menampilkan daftar video yang relevan dengan kueri Anda. Aplikasi juga akan memberikan tautan ke bagian dalam video di mana jawaban atas pertanyaan tersebut berada.

Berikut beberapa contoh kueri yang bisa Anda coba:

- Apa itu Azure Machine Learning?
- Bagaimana cara kerja convolutional neural networks?
- Apa itu neural network?
- Bisakah saya menggunakan Jupyter Notebooks dengan Azure Machine Learning?
- Apa itu ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berupaya untuk memberikan terjemahan yang akurat, harap diketahui bahwa terjemahan otomatis dapat mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang berwenang. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa terjemahan profesional oleh manusia. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang timbul akibat penggunaan terjemahan ini.
